In [1]:
import pandas as pd
import gzip
import json
from pathlib import Path
import tqdm

raf_path = Path('../2-9.playwright/output/raf/')
imr = Path('./imr')

imr.mkdir(parents=True, exist_ok=True)

def deal(path):
    with gzip.open(path, 'rt') as fp:
        ret = json.load(fp)
    ret['raf_path'] = path.as_posix()
    idx = path.as_posix().split('/')[-1].split('.')[0]
    try:
        ret['idx'] = int(idx)
    except ValueError:
        ret['idx'] = idx
    return ret

In [2]:
# 0413 + 0504_czy = 0504-df_label
df_label = pd.read_pickle('./output/0501-df_label-raf.pkl.zst.bak', compression='zstd')
df_label

,url,category,label_ERROR,label_usecase,label_interactions,label_comments
0,http://0ldsk00l.ca/,['bq'],0,1,0,
1,http://1000.chromeexperiments.com,[awwwards],0,1,1,
2,http://1000000-lives.com/,['bq'],0,3,0,
3,http://10k.global/,['bq'],0,1,1,
4,http://123screenprotector.com/,['bq'],0,3,0,
...,...,...,...,...,...,...
29237,https://map.e-mangistau.kz/,['bq'],1,<NA>,<NA>,<NA>
29238,https://map.e-zhetisu.kz/,['bq'],0,1,1,
29239,https://map.iturkistan.kz/,['bq'],1,<NA>,<NA>,<NA>
29240,https://qrg.geoportal.kz/,['bq'],1,<NA>,<NA>,<NA>


## RAF

In [ ]:
if Path('imr/0504-df_raw_all-raf.pkl.zst').exists():
    df = pd.read_pickle('imr/0504-df_raw_all-raf.pkl.zst', compression='zstd')
else:
    df = pd.DataFrame([deal(p) for p in tqdm.tqdm(sorted(raf_path.glob("*.gz")))]).set_index('idx')
    df = df_label.drop(columns=['url']).join(df)
    df.to_pickle('imr/0504-df_raw_all-raf.pkl.zst', compression='zstd')

In [4]:
if Path('imr/0504-df_frame-raf.pkl.zst').exists():
    df_frame = pd.read_pickle('imr/0504-df_frame-raf.pkl.zst', compression='zstd')
else:
    df.dropna(subset=['frame'], inplace=True)
    df_frame = df.drop(columns=['frame', 'events_time_hp']).join(pd.DataFrame(df['frame'].to_list(), index=df.index)['gl_rafs'].explode()).reset_index(drop=False).rename(columns={'index': 'idx'})
    df_frame = df_frame.drop(columns=['gl_rafs']).join(pd.DataFrame(df_frame['gl_rafs'].str['value'].to_list(), index=df_frame.index)['data'])
    df_frame = df_frame.drop(columns=['data']).join(pd.json_normalize(df_frame['data']).add_prefix('frame.'), how='right').dropna(subset=['frame.counter.usedWebGL']) #.drop_duplicates(subset=['frame.url']).reset_index(drop=True)
    df_frame.to_pickle('imr/0504-df_frame-raf.pkl.zst', compression='zstd')

In [5]:
if Path('imr/0504-df_context-raf.pkl.zst').exists():
    df_context = pd.read_pickle('imr/0504-df_context-raf.pkl.zst', compression='zstd')
else:
    df_context = df_frame.explode('frame.contextInfo').reset_index(drop=False).rename(columns={'index': 'idx-frame'})
    df_context = df_context.drop(columns=['frame.contextInfo']).join(
        pd.json_normalize(df_context['frame.contextInfo']).add_prefix('frame.context.'))
    df_context.to_pickle('imr/0504-df_context-raf.pkl.zst', compression='zstd')

In [6]:
if Path('imr/0504-df_context-noraf.pkl.zst').exists():
    df_context_noraf = pd.read_pickle('imr/0504-df_context-noraf.pkl.zst', compression='zstd')
else:
    df_context_noraf = df_context.drop(columns=['frame.context.rafList'])
    df_context_noraf.to_pickle('imr/0504-df_context-noraf.pkl.zst', compression='zstd')

In [7]:
if Path('imr/0504-df_shader-noraf.pkl.zst').exists():
    df_shader = pd.read_pickle('imr/0504-df_shader-noraf.pkl.zst', compression='zstd')
else:
    df_shader = df_context_noraf[['idx-frame','idx','category','label_ERROR','label_usecase','label_interactions','label_comments','url','date','raf_path','frame.url','frame.title','frame.date','frame.when','frame.context.maghsk.shaders',]].explode('frame.context.maghsk.shaders').dropna(subset=['frame.context.maghsk.shaders']).reset_index(drop=False).rename(columns={'index': 'idx-context'})
    df_shader = df_shader.drop(columns='frame.context.maghsk.shaders').join(pd.json_normalize(df_shader['frame.context.maghsk.shaders']).add_prefix('frame.context.maghsk.shaders.')).dropna(subset=['frame.context.maghsk.shaders.source'])
    df_shader['frame.context.maghsk.shaders.type'] = df_shader['frame.context.maghsk.shaders.type'].astype(str).replace('35632', 'frag').replace('35633', 'vert')
    df_shader['frame.context.maghsk.shaders.length'] = df_shader['frame.context.maghsk.shaders.source'].map(lambda x: len(x))
    df_shader.to_pickle('imr/0504-df_shader-noraf.pkl.zst', compression='zstd')

## Spector

In [10]:
import base64
import json
import tqdm
import pandas as pd
from pathlib import Path
import gzip


In [11]:
spec_path = Path('../2-9.playwright/output/spector-one/')

In [12]:
def spector_deal(p):
    ret = json.loads(p.read_text())
    ret['spec_path'] = p.as_posix()
    frame = []
    for spector in ret['spector']:
        try:
            frame.append({"url": spector['value']['url'], "data": [base64.b64decode(x[22:]) for x in spector['value']['data']]})
        except:
            pass
    del ret['spector']
    ret['frame'] = frame
    return ret

In [13]:
if Path('imr/0504-df_spector.pkl.zst').exists():
    df = pd.read_pickle('imr/0504-df_spector.pkl.zst', compression='zstd')
else:
    df = pd.DataFrame([spector_deal(p) for p in tqdm.tqdm(sorted(spec_path.glob("*.json")))]).set_index('idx')
    df = df.explode('frame').dropna(subset=['frame']).reset_index(drop=False).rename(columns={'index': 'idx'})
    df = df.drop(columns='frame').join(pd.json_normalize(df['frame']).add_prefix('frame.')).explode('frame.data').dropna(subset='frame.data').reset_index(drop=False).rename(columns={'index': 'idx-frame'})
    df['frame.data'] = df['frame.data'].map(lambda x: json.loads(gzip.decompress(x)))
    df.to_pickle('imr/0504-df_spector.pkl.zst', compression='zstd')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 29242/29242 [00:43<00:00, 669.77it/s]


## Simple

In [12]:
simple_path = Path('../2-9.playwright/output/simple/')

In [13]:
def deal_simple(p):
    with gzip.open(p, 'rt') as f:
        return json.load(f)

In [14]:
if Path('imr/0504-df_simple.pkl.zst').exists():
    df = pd.read_pickle('imr/0504-df_simple.pkl.zst', compression='zstd')
else:
    df_raw = pd.DataFrame([deal_simple(p) for p in tqdm.tqdm(sorted(simple_path.glob("*.gz")))])
    df = df_raw.drop(columns=['frame', 'events_time_hp']).join(pd.json_normalize(df_raw['frame']).drop(columns=['net_idle_counters','gl_simple_counters'])).explode('gl_simples').dropna(subset='gl_simples').reset_index(drop=False).rename(columns={'index': 'idx'})
    df = df.drop(columns='gl_simples').join(pd.json_normalize(df['gl_simples'].str['value']).drop(columns=['data', 'error.name']).add_prefix('frame.')).explode('frame.data.contextInfo').dropna(subset=['frame.data.contextInfo']).reset_index(drop=False).rename(columns={'index': 'idx-frame'})
    df = df.drop(columns=['frame.data.contextInfo']).join(pd.json_normalize(df['frame.data.contextInfo']).add_prefix('frame.data.contextInfo.'))
    df.to_pickle('imr/0504-df_simple.pkl.zst', compression='zstd')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 29241/29241 [00:22<00:00, 1276.63it/s]


## Worker

In [8]:
import gzip
import json

In [9]:
def deal_worker(p):
    with gzip.open(p, 'rt') as f:
        return json.load(f)
        

In [10]:
deal_worker('../2-9.playwright/output/simple/00000.json.gz')

{'url': 'http://0ldsk00l.ca/',
 'date': 1683039873090,
 'events_time_hp': {'start_time_hp': 2221.028041958809,
  'net_idle_time_hp': 3057.1718759536743,
  'gl_simple_time_hp': 18080.846422970295},
 'frame': {'net_idle_counters': [{'status': 'fulfilled',
    'value': {'name': '',
     'url': 'http://0ldsk00l.ca/',
     'data': {'contextsNum': 1,
      'usedWebGL': 372.7999999523163,
      'usedOffScreenCanvas': True},
     'error': None}}],
  'gl_simple_counters': [{'status': 'fulfilled',
    'value': {'name': '',
     'url': 'http://0ldsk00l.ca/',
     'data': {'contextsNum': 1,
      'usedWebGL': 372.7999999523163,
      'usedOffScreenCanvas': True},
     'error': None}}],
  'gl_simples': [{'status': 'fulfilled',
    'value': {'name': '',
     'url': 'http://0ldsk00l.ca/',
     'data': {'url': 'http://0ldsk00l.ca/',
      'title': '0ldsk00l',
      'accessTime': 1683039888988,
      'usedOffScreenCanvas': True,
      'when': 15891.799999952316,
      'whenUsedWebgl': 372.7999999523163

In [ ]:
worker_path = Path('../2-9.playwright/output/simple/'):
df_raw = pd.DataFrame([deal_simple(p) for p in tqdm.tqdm(sorted(worker_path.glob("*.gz")))])